In [ ]:
import numpy as np
from ipywidgets import IntSlider, interact, ToggleButtons, HBox, VBox, Output
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, Slider, Label
from bokeh.layouts import widgetbox
from bokeh.models.widgets import RadioButtonGroup, Select
from bokeh.layouts import column, row
from bokeh.models import NumeralTickFormatter, BasicTickFormatter, Range1d, Span, Arrow, OpenHead, NormalHead, VeeHead
from bokeh.io import push_notebook
output_notebook()

p = figure(height=400, width=450, background_fill_color="lightgray")
p.x_range = Range1d(1500,1600)
p.outline_line_width = 1; p.outline_line_color = "black"; p.min_border_top = 10
p.xaxis.axis_label = "Wavelength (nm)"
p.yaxis.axis_label = "MZI Transmission"
p.xaxis.axis_label_text_font_size = "12pt"
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

wl=np.linspace(1400,1700,301)/1000 # wavelength in micron
neff = 2.43
L_up=100 # Length of upper MZM arm in micron
delta_L=25 # Additional length in lower arm
L_low=L_up+delta_L

delta_n=np.linspace(0,6e-4,100)
T=np.absolute(np.exp(1j*2*np.pi*neff*L_up/wl)+np.exp(1j*2*np.pi*neff*L_low/wl))**2/4*0.85
l1=p.line(wl*1000,T,line_color='darkblue')

xl=(1/(1/1.519+1/(4*2.43*25)))*1000
xr=(1/(1/1.519-1/(4*2.43*25)))*1000
FWHM = Arrow(start=VeeHead(size=10), end=VeeHead(size=10), line_color="red", line_dash='dashed',
                   x_start=xl, y_start=0.425, x_end=xr, y_end=0.425)
p.add_layout(FWHM)
label1=Label(x=1512, y=0.36, text='FWHM', text_color='red', background_fill_color='lightgray')
p.add_layout(label1)
xrFSR=(1/(1/1.519-1/(2.43*25)))*1000
FSR = Arrow(start=VeeHead(size=10), end=VeeHead(size=10), line_color="red", line_dash='dashed',
                   x_start=1519, y_start=0.85, x_end=xrFSR, y_end=0.85)
p.add_layout(FSR)
label2=Label(x=(1519+xrFSR)/2-4, y=0.79, text='FSR', text_color='red', background_fill_color='lightgray')
p.add_layout(label2)

L_up_slider =  IntSlider(min=80,max=110,step=5,value=100,description='Upper Arm Length (micron)',
                         style = {'description_width': '200px'}, layout={'width': '450px'})
L_low_slider = IntSlider(min=115,max=150,step=5,value=125,description='Lower Arm Length (micron)',
                         style = {'description_width': '200px'}, layout={'width': '450px'})

def replot(L_up,L_low):
    T=np.absolute(np.exp(1j*2*np.pi*neff*L_up/wl)+np.exp(1j*2*np.pi*neff*L_low/wl))**2/4*0.85
    l1.data_source.data['y']=T
    xl=(1/(1/1.519+1/(4*2.43*(L_low-L_up))))*1000
    xr=(1/(1/1.519-1/(4*2.43*(L_low-L_up))))*1000
    FWHM.x_start=xl
    FWHM.x_end=xr
    xrFSR=(1/(1/1.519-1/(2.43*(L_low-L_up))))*1000
    FSR.x_end=xrFSR
    label2.x=(1519+xrFSR)/2-4
    push_notebook(handle=fig_handle)

fig_handle = show(p,notebook_handle=True)
interact(replot, L_up=L_up_slider, L_low=L_low_slider);